# How to pass tool outputs to chat models

Some models are capable of tool calling-generating arguments that conform to a specific user-provided schema. This guide will demonstrate how to use those tool calls to actually call a function and properly pass the results back to the model.

![img](../asset/tool_invocation.png)

First, let's define our tools and our model.


In [1]:
import os
import yaml
from langchain_community.chat_models import ChatZhipuAI


with open('../utils/config.yml', 'r') as stream:
    zhipuai_api_key = yaml.safe_load(stream)['api_key']

os.environ['ZHIPUAI_API_KEY'] = zhipuai_api_key

chat_model = ChatZhipuAI(
    model='glm-4-plus',
    temperature=0,
)

In [2]:
from langchain_core.tools import tool

@tool
def add(a:int, b:int):
    """Adds a and b."""
    return a + b

@tool
def multiply(a:int, b:int):
    """Multiply a and b."""
    return a * b

tools=[add, multiply]

llm_with_tools = chat_model.bind_tools(tools)

In [3]:
from langchain_core.messages import HumanMessage

query = "What's 3 * 12? Also, what is 11 + 49? "

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)
print(ai_msg.tool_calls)
messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_9021306401513663733', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_9021306401513663734', 'type': 'tool_call'}]


Next let's invoke the tool functions using the args the model populated!

Conveniently, if we invoke a LangChain `Tool` with a `ToolCall`, we'll automatically get back a `ToolMessage` that can be feedback to the model:

In [4]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {'add': add, 'multiply': multiply}[tool_call['name'].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

[HumanMessage(content="What's 3 * 12? Also, what is 11 + 49? "),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'id': 'call_9021306401513663733', 'index': 0, 'type': 'function'}, {'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'id': 'call_9021306401513663734', 'index': 1, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 282, 'total_tokens': 312}, 'model_name': 'glm-4-plus', 'finish_reason': 'tool_calls'}, id='run-ea205cb4-8e78-47ee-8734-36b29b20fedf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_9021306401513663733', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_9021306401513663734', 'type': 'tool_call'}]),
 ToolMessage(content='36', name='multiply', tool_call_id='call_9021306401513663733'),
 ToolMessage(content='60', name='add', tool_call_id='call_9021306401513663734')]

And finally, we'll invoke the model with the tool results. The model will use this information to generate a final answer to our original query:

In [5]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of 3 multiplied by 12 is 36, and the sum of 11 and 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 290, 'total_tokens': 318}, 'model_name': 'glm-4-plus', 'finish_reason': 'stop'}, id='run-494c8b1d-0abb-445a-ba0e-b0a30a0f39c3-0')

Note that each `ToolMessage` must include a `tool_call_id`  that matches an `id` in the original tool calls that the model generates. This helps the model match tool response with tool calls.

Tool calling agents, like those in LangGraph, use this basic flow to answer queries and solve tasks.